In [1]:
import sys
sys.path.append('/home/is/akiyoshi-n/my-project')

In [2]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_multiclass_dataset, load_dataset_2class_classification, load_dataset_4class_Multi_classification, split_test_data_stratify, load_text_dataset, split_multilabel_data
from src.my_project.train_v2 import MultiClassClassifier
from sklearn.model_selection import train_test_split
from src.my_project.dataset import load_multiclass_dataset
import wandb
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [3]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))
# モデル出力先ディレクトリ
output_model_dir = Path('/home/is/akiyoshi-n/my-project/outputs_model')

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 4
# 乱数シード
SEED = 2024
# クラス数
NUM_LABELS = 4
# 閾値
THRESH = 0.5

### 2class

In [5]:
# 辞書型でデータ取得
data = load_dataset_2class_classification(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")
data_pred = load_dataset_2class_classification(f"{DATASET_PATH}/raw_lifestory_GPT-4o_annotation.xlsx")

In [15]:
answer = np.array(data['labels'])
pred = np.array(data_pred['labels'])
accuracy = accuracy_score(y_true=answer, y_pred=pred)
f1 = f1_score(y_true=answer, y_pred=pred, average='macro')
print({"accuracy": accuracy, "macro_f1": macro_f1})

{'accuracy': 0.7017543859649122, 'macro_f1': 0.6262990108295193}


### 4class

In [5]:
# データの読み込み
data, class_name = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [6]:
# データの読み込み
data_pred, class_name_pred = load_dataset_4class_Multi_classification(f"{DATASET_PATH}/raw_lifestory_GPT-4o_annotation.xlsx")

In [11]:
# AccuracyとMacro-F1
# numpy.ndarray配列に変換
answer = np.array(data['labels'])
pred = np.array(data_pred['labels'])
# accuracyを計算
accuracy = accuracy_score(y_true=answer, y_pred=pred)
# macro f1を計算
macro_f1 = f1_score(y_true=answer, y_pred=pred, average='macro', zero_division=0)
# クラス毎のF1値を計算
class_f1 = [round(score, 3) for score in f1_score(y_true=answer, y_pred=pred, average=None, zero_division=0)]
print({"accuracy": accuracy, "macro_f1": macro_f1, "class_f1": class_f1})

{'accuracy': 0.5727554179566563, 'macro_f1': 0.6262990108295193, 'class_f1': [0.646, 0.559, 0.71, 0.591]}


### 21class

In [6]:
# データの読み込み
data, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [7]:
class_name
# 喫煙が抜けてる．

array(['1 - A. 睡眠', '1 - B. 身の回りの用事', '1 - C - a. 食事-飲酒あり',
       '1 - C - b. 食事-飲酒なし・不明', '2 - A - a. 通勤・通学', '2 - A - b. その他の移動',
       '2 - B. 仕事', '2 - C. 学業', '2 - D. 家事', '2 - E. 介護・看護', '2 - F. 育児',
       '3 - A. メディア', '3 - B. 買い物', '3 - C - a. 趣味・娯楽-体動かす',
       '3 - C - b. 趣味・娯楽-体動かさない', '3 - D - a. 交際-物理的接触',
       '3 - D - b. 交際-オンライン的接触', '3 - E. 受診・療養', '3 - F - b. その他の活動',
       '4. 行動なし'], dtype=object)

In [20]:
# データの読み込み
data_pred, class_name_pred = load_multiclass_dataset(f"{DATASET_PATH}/raw_lifestory_GPT-4o_annotation.xlsx")

In [ ]:
for sublist in data['labels']:
    sublist.insert(-2, 0)

In [20]:
len(data['labels'][0])

21

In [21]:
# AccuracyとMacro-F1
# numpy.ndarray配列に変換
answer = np.array(data['labels'])
pred = np.array(data_pred['labels'])
# accuracyを計算
accuracy = accuracy_score(y_true=answer, y_pred=pred)
# macro f1を計算
macro_f1 = f1_score(y_true=answer, y_pred=pred, average='macro', zero_division=0)
# クラス毎のF1値を計算
class_f1 = [round(score, 3) for score in f1_score(y_true=answer, y_pred=pred, average=None, zero_division=0)]
print({"accuracy": accuracy, "macro_f1": macro_f1, "class_f1": class_f1})

{'accuracy': 0.5036119711042312, 'macro_f1': 0.5283847855501879, 'class_f1': [0.714, 0.591, 0.706, 0.659, 0.762, 0.602, 0.561, 0.0, 0.348, 0.311, 0.554, 0.774, 0.731, 0.629, 0.636, 0.471, 0.468, 0.615, 0.0, 0.374, 0.591]}


### カッパ係数

In [107]:
# データの読み込み
# data_Nakae, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")
data_Nakae, class_name = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_nakae_annotation.xlsx")

In [108]:
class_name
# 喫煙が抜けてる．

array(['1 - A. 睡眠', '1 - B. 身の回りの用事', '1 - C - a. 食事-飲酒あり',
       '1 - C - b. 食事-飲酒なし・不明', '2 - A - a. 通勤・通学', '2 - A - b. その他の移動',
       '2 - B. 仕事', '2 - C. 学業', '2 - D. 家事', '2 - E. 介護・看護', '2 - F. 育児',
       '3 - A. メディア', '3 - B. 買い物', '3 - C - a. 趣味・娯楽-体動かす',
       '3 - C - b. 趣味・娯楽-体動かさない', '3 - D - a. 交際-物理的接触',
       '3 - D - b. 交際-オンライン的接触', '3 - E. 受診・療養', '3 - F - b. その他の活動',
       '4. 行動なし'], dtype=object)

In [109]:
# データの読み込み
data_Akiyoshi, class_name_pred = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_annotator_annotation.xlsx")
# data_Akiyoshi, class_name_pred = load_multiclass_dataset(f"{DATASET_PATH}/lifestory_GPT+human_annotation.xlsx")

In [110]:
class_name

array(['1 - A. 睡眠', '1 - B. 身の回りの用事', '1 - C - a. 食事-飲酒あり',
       '1 - C - b. 食事-飲酒なし・不明', '2 - A - a. 通勤・通学', '2 - A - b. その他の移動',
       '2 - B. 仕事', '2 - C. 学業', '2 - D. 家事', '2 - E. 介護・看護', '2 - F. 育児',
       '3 - A. メディア', '3 - B. 買い物', '3 - C - a. 趣味・娯楽-体動かす',
       '3 - C - b. 趣味・娯楽-体動かさない', '3 - D - a. 交際-物理的接触',
       '3 - D - b. 交際-オンライン的接触', '3 - E. 受診・療養', '3 - F - b. その他の活動',
       '4. 行動なし'], dtype=object)

In [111]:
# 100行目までのデータを取得
data_Nakae_100 = data_Nakae['labels'][0:100]
data_Akiyoshi_100 = data_Akiyoshi['labels'][0:100]
data_Nakae_100_200 = data_Nakae['labels'][100:500]
data_Akiyoshi_100_200 = data_Akiyoshi['labels'][100:500]

In [112]:
data_labels_np = np.array(data_Nakae_100)
data_labels_np.sum(axis=0)

array([ 3,  3,  1,  3,  0,  5,  5,  1,  4,  2,  2,  4, 10,  6,  3,  5,  1,
        8, 13, 40])

In [113]:
data_labels_np = np.array(data_Akiyoshi_100)
data_labels_np.sum(axis=0)

array([ 2,  4,  1,  5,  0,  7,  3,  1,  9,  5,  3,  4, 12,  1,  4,  3,  1,
        9,  0,  8, 35])

In [114]:
data_labels_np = np.array(data_Nakae_100_200)
data_labels_np.sum(axis=0)

array([  2,   6,   0,   6,   5,  21,  15,   0,  25,   6,  11,  20,  19,
         6,   8,   9,   7,  17,  13, 215])

In [115]:
print(np.array(data_Nakae_100).shape)
print(np.array(data_Akiyoshi_100).shape)
print(np.array(data_Nakae_100_200).shape)
print(np.array(data_Akiyoshi_100_200).shape)

(100, 20)
(100, 21)
(400, 20)
(400, 21)


In [116]:
for sublist in data_Nakae_100:
    sublist.insert(-2, 0)

In [117]:
for sublist in data_Nakae_100_200:
    sublist.insert(-2, 0)

In [88]:
print(np.array(data_Nakae_100).shape)
print(np.array(data_Akiyoshi_100).shape)

(100, 21)
(100, 21)


In [89]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

In [90]:
# NumPy配列に変換（必要に応じて）
data_Nakae_100 = np.array(data_Nakae_100)
data_Akiyoshi_100 = np.array(data_Akiyoshi_100)

# 各ラベル列ごとにCohen's Kappaを計算（バリエーションがゼロの場合スキップ）
kappa_scores = []
for i in range(data_Nakae_100.shape[1]):
    if np.var(data_Nakae_100[:, i]) == 0 and np.var(data_Akiyoshi_100[:, i]) == 0:
        print(f"Label {i + 1}: Skipped due to no variance.")
        kappa_scores.append(1.0)  # 完全一致とみなす場合
        continue
    kappa = cohen_kappa_score(data_Nakae_100[:, i], data_Akiyoshi_100[:, i])
    kappa_scores.append(kappa if not np.isnan(kappa) else 0)

# 平均Kappa
average_kappa = np.nanmean(kappa_scores)  # NaNを無視して平均計算
print(f"Average Cohen's Kappa: {average_kappa:.2f}")

Label 5: Skipped due to no variance.
Label 19: Skipped due to no variance.
Average Cohen's Kappa: 0.67


In [91]:
# 行が完全一致している数を計算
exact_matches = np.sum(np.all(data_Nakae_100 == data_Akiyoshi_100, axis=1))
# 行数
num_rows = data_Nakae_100.shape[0]
# 完全一致率
exact_match_rate = exact_matches / num_rows
print(f"Exact match rate: {exact_match_rate:.2%}")


Exact match rate: 65.00%


In [118]:
# numpy配列に変換
data_Nakae_100_200 = np.array(data_Nakae_100_200)
data_Akiyoshi_100_200 = np.array(data_Akiyoshi_100_200)

# 各ラベル列ごとにCohen's Kappaを計算（バリエーションがゼロの場合スキップ）
kappa_scores = []
for i in range(data_Nakae_100_200.shape[1]):
    if np.var(data_Nakae_100_200[:, i]) == 0 and np.var(data_Akiyoshi_100_200[:, i]) == 0:
        print(f"Label {i + 1}: Skipped due to no variance.")
        kappa_scores.append(1.0)  # 完全一致とみなす場合
        continue
    kappa = cohen_kappa_score(data_Nakae_100_200[:, i], data_Akiyoshi_100_200[:, i])
    kappa_scores.append(kappa if not np.isnan(kappa) else 0)

# 平均Kappa
average_kappa = np.nanmean(kappa_scores)  # NaNを無視して平均計算
print(f"Average Cohen's Kappa: {average_kappa:.2f}")

Average Cohen's Kappa: 0.45


In [119]:
# 行が完全一致している数を計算
exact_matches = np.sum(np.all(data_Nakae_100_200 == data_Akiyoshi_100_200, axis=1))
# 行数
num_rows = data_Nakae_100_200.shape[0]
# 完全一致率
exact_match_rate = exact_matches / num_rows
print(f"Exact match rate: {exact_match_rate:.2%}")

Exact match rate: 48.25%
